In [1]:
!pip install openvino-dev


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras

In [4]:
# Read the data from file
df = pd.read_csv("./data/battery_data.csv")

print(df)

                             timestamp  batteryId  ambientTemp  \
0     2025-10-06 11:32:27.639000+00:00          1        20.61   
1     2025-10-06 11:32:30.640000+00:00          1        20.47   
2     2025-10-06 11:32:33.639000+00:00          1        20.22   
3     2025-10-06 11:32:36.639000+00:00          1        20.47   
4     2025-10-06 11:32:39.639000+00:00          1        19.98   
...                                ...        ...          ...   
1195  2025-10-06 12:32:12.638000+00:00          1         0.52   
1196  2025-10-06 12:32:15.638000+00:00          1         0.91   
1197  2025-10-06 12:32:18.639000+00:00          1         0.98   
1198  2025-10-06 12:32:21.638000+00:00          1         0.78   
1199  2025-10-06 12:32:24.639000+00:00          1         0.35   

      batteryCurrent  batteryTemp  batteryVoltage  currentLoad  distance  \
0              17.25        25.04           47.17        100.0      6.53   
1              19.50        25.04           47.06      

In [5]:
# Define stress condition (1 = Stress, 0 = Normal)
def detect_stress(row):
    if row["batteryCurrent"] > 400 or row["batteryTemp"] > 50 or row["stateOfCharge"] < 0.05 or row["batteryVoltage"] < 320:
        return 1  # Stress condition
    return 0  # Normal condition


# Apply stress detection
df["stressIndicator"] = df.apply(detect_stress, axis=1)

print(df)

                             timestamp  batteryId  ambientTemp  \
0     2025-10-06 11:32:27.639000+00:00          1        20.61   
1     2025-10-06 11:32:30.640000+00:00          1        20.47   
2     2025-10-06 11:32:33.639000+00:00          1        20.22   
3     2025-10-06 11:32:36.639000+00:00          1        20.47   
4     2025-10-06 11:32:39.639000+00:00          1        19.98   
...                                ...        ...          ...   
1195  2025-10-06 12:32:12.638000+00:00          1         0.52   
1196  2025-10-06 12:32:15.638000+00:00          1         0.91   
1197  2025-10-06 12:32:18.639000+00:00          1         0.98   
1198  2025-10-06 12:32:21.638000+00:00          1         0.78   
1199  2025-10-06 12:32:24.639000+00:00          1         0.35   

      batteryCurrent  batteryTemp  batteryVoltage  currentLoad  distance  \
0              17.25        25.04           47.17        100.0      6.53   
1              19.50        25.04           47.06      

In [6]:
# Count the number of stress events
stress_count = df["stressIndicator"].sum()

print(f"Total stress events detected: {stress_count}")

Total stress events detected: 1200


In [7]:
# Define Features and Target
features = ["stateOfCharge", "stateOfHealth", "batteryCurrent", "batteryVoltage", "kmh", "distance", "batteryTemp", "ambientTemp", "currentLoad"]
X = df[features]
y = df["stressIndicator"]

In [8]:
# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [9]:
# Define neural network
mlp_tf = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile model
mlp_tf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

2025-10-06 12:35:48.576899: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-10-06 12:35:48.576943: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:137] retrieving CUDA diagnostic information for host: model-training-0
2025-10-06 12:35:48.576954: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:144] hostname: model-training-0
2025-10-06 12:35:48.577110: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:168] libcuda reported version is: 580.82.7
2025-10-06 12:35:48.577137: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:172] kernel reported version is: 580.82.7
2025-10-06 12:35:48.577145: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:259] kernel version seems to match DSO: 580.82.7


In [10]:
# Train model
mlp_tf.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=2)

Epoch 1/50
27/27 - 1s - 49ms/step - accuracy: 0.7477 - loss: 0.5452 - val_accuracy: 1.0000 - val_loss: 0.2965
Epoch 2/50
27/27 - 0s - 5ms/step - accuracy: 1.0000 - loss: 0.1804 - val_accuracy: 1.0000 - val_loss: 0.0986
Epoch 3/50
27/27 - 0s - 5ms/step - accuracy: 1.0000 - loss: 0.0596 - val_accuracy: 1.0000 - val_loss: 0.0379
Epoch 4/50
27/27 - 0s - 5ms/step - accuracy: 1.0000 - loss: 0.0252 - val_accuracy: 1.0000 - val_loss: 0.0193
Epoch 5/50
27/27 - 0s - 5ms/step - accuracy: 1.0000 - loss: 0.0139 - val_accuracy: 1.0000 - val_loss: 0.0119
Epoch 6/50
27/27 - 0s - 4ms/step - accuracy: 1.0000 - loss: 0.0088 - val_accuracy: 1.0000 - val_loss: 0.0080
Epoch 7/50
27/27 - 0s - 4ms/step - accuracy: 1.0000 - loss: 0.0061 - val_accuracy: 1.0000 - val_loss: 0.0058
Epoch 8/50
27/27 - 0s - 4ms/step - accuracy: 1.0000 - loss: 0.0044 - val_accuracy: 1.0000 - val_loss: 0.0044
Epoch 9/50
27/27 - 0s - 5ms/step - accuracy: 1.0000 - loss: 0.0034 - val_accuracy: 1.0000 - val_loss: 0.0034
Epoch 10/50
27/27 

In [15]:
# Save model as keras
os.makedirs("models")
mlp_tf.save("models/battery_stress_model.keras")
print("Model saved as TensorFlow format in 'models/battery_stress_model.keras'")

Model saved as TensorFlow format in 'models/battery_stress_model.keras'


In [16]:
# Load model
model = tf.keras.models.load_model("models/battery_stress_model.keras")

# Export SavedModel
tf.saved_model.save(model, "models/battery_stress_model")

print("Model converted to SavedModel format")

INFO:tensorflow:Assets written to: models/battery_stress_model/assets


INFO:tensorflow:Assets written to: models/battery_stress_model/assets


Model converted to SavedModel format


In [17]:
!mo --saved_model_dir models/battery_stress_model --output_dir models/battery_stress_model

[ INFO ] MO command line tool is considered as the legacy conversion API as of OpenVINO 2023.2 release.
In 2025.0 MO command line tool and openvino.tools.mo.convert_model() will be removed. Please use OpenVINO Model Converter (OVC) or openvino.convert_model(). OVC represents a lightweight alternative of MO and provides simplified model conversion API. 
Find more information about transition from MO to OVC at https://docs.openvino.ai/2023.2/openvino_docs_OV_Converter_UG_prepare_model_convert_model_MO_OVC_transition.html
2025-10-06 12:38:28.063393: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759754308.090207    6196 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759754308.098450    6196 cuda_blas.cc:1418] Unable to register cuBLAS factory